In [2]:
from pyspark.sql import SparkSession

# (8 cores, 16gb per machine) x 5 = 40 cores

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.87:7077") \
        .appName("Part_B_Raheel_Ali")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .getOrCreate()


# Old API (RDD)
spark_context = spark_session.sparkContext

In [13]:
### B.1
dataSet = spark_session.read\
    .option("header", "true")\
    .csv("hdfs://192.168.2.87:9000/parking-citations.csv")\
    .cache()

if(dataSet != 0):
    dataSet.show()    

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    null|       null|            CA|           200304|null|HOND|        PA|  

In [14]:
### B.2
dataSet.count()

9881842

In [15]:
### B.3
dataSet.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [16]:
### B.4
dataSet.rdd.getNumPartitions()

11

In [17]:
### B.5
columnsToDrop = ['Agency Description', 'Agency', 'Route']
dataSet = dataSet.drop(*columnsToDrop)

In [18]:
### B.5
dataSet.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [ ]:
### B.6
# from pyspark.sql.types import IntegerType
# dataSet2 = dataSet.withColumn("Fine amount", data_frame["Fine amount"].cast(IntegerType()))
# max_fine = data_frame_2.groupby('Fine amount')\
#       .count().orderBy('Fine amount', ascending=False)
# max_fine.take(30)

In [24]:
### B.7
vehiclesToShow = dataSet.groupby('Make')\
      .count().orderBy('count', ascending=False)
vehiclesToShow.take(10)

[Row(Make='TOYT', count=1633266),
 Row(Make='HOND', count=1113834),
 Row(Make='FORD', count=860828),
 Row(Make='NISS', count=709250),
 Row(Make='CHEV', count=674422),
 Row(Make='BMW', count=450909),
 Row(Make='MERZ', count=402126),
 Row(Make='VOLK', count=335618),
 Row(Make='HYUN', count=304934),
 Row(Make='DODG', count=290979)]

In [25]:
# release the cores for another application!
spark_context.stop()